In [1]:
pip install pybaseball --upgrade pybaseball

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\miraj\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import pybaseball as pyb
from itertools import permutations
import matplotlib.pyplot as plt
from scipy.stats import multinomial
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')

In [14]:
def figure_outcomes(player_stats):
  return 0
def get_all_players_with_teams(season=2024):
    """Get all players with their team affiliations."""
    try:
        return pyb.batting_stats(season, qual=150)
    except Exception as e:
        print(f"Error retrieving batting stats for {season}: {e}")
        return pd.DataFrame()

def filter_by_team(df, team_abbr):
    """Filter DataFrame by team."""
    team_cols = [col for col in df.columns if 'team' in col.lower() or 'tm' in col.lower()]
    if team_cols:
        return df[df[team_cols[0]] == team_abbr]
    print(f"No team column found in DataFrame.")
    return pd.DataFrame()

def aggregate_team(df):
    """Aggregate players by team."""
    teams = ['ARI', 'ATL', 'BAL', 'BOS', 'CHC', 'CWS', 'CIN', 'CLE',
             'COL', 'DET', 'HOU', 'KC', 'LAA', 'LAD', 'MIA', 'MIL',
             'MIN', 'NYM', 'NYY', 'OAK', 'PHI', 'PIT', 'SD', 'SF',
             'SEA', 'STL', 'TB', 'TEX', 'TOR', 'WSN']
    team_rosters = {}
    for team in teams:
        team_df = filter_by_team(df, team)
        if not team_df.empty:
            team_rosters[team] = team_df
    return team_rosters
test_df = get_all_players_with_teams(2025)
players = aggregate_team(test_df)
NYY_players = filter_by_team(test_df, 'NYY')


In [ ]:
#Using this for reference point

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,...,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA,L-WAR
0,15640,2025,Aaron Judge,NYY,33,132,479,590,154,81,...,118.1,190,0.560,339,0.151,0.298,0.302,0.698,0.454,7.8
65,15998,2025,Cody Bellinger,NYY,29,132,513,567,141,86,...,110.2,175,0.392,446,0.160,0.234,0.262,0.446,0.331,4.1
53,20454,2025,Jazz Chisholm Jr.,NYY,27,110,392,456,94,52,...,110.9,119,0.442,269,0.153,0.294,0.246,0.524,0.369,3.9
46,18564,2025,Trent Grisham,NYY,28,125,433,504,106,66,...,109.8,146,0.455,321,0.200,0.270,0.265,0.521,0.383,3.2
248,27562,2025,Austin Wells,NYY,25,110,354,398,76,36,...,111.9,120,0.449,267,0.133,0.260,0.216,0.409,0.296,2.4
84,29576,2025,Ben Rice,NYY,26,120,404,462,98,52,...,113.5,173,0.553,313,0.181,0.274,0.292,0.568,0.406,2.2
2,4949,2025,Giancarlo Stanton,NYY,35,59,188,212,54,29,...,118.0,68,0.548,124,0.192,0.329,0.262,0.560,0.379,1.9
169,9218,2025,Paul Goldschmidt,NYY,37,128,444,484,123,82,...,111.2,150,0.419,358,0.173,0.260,0.274,0.465,0.337,1.2
319,27647,2025,Anthony Volpe,NYY,24,139,495,550,103,51,...,110.5,156,0.422,370,0.187,0.288,0.237,0.424,0.315,0.6
195,28080,2025,Jasson Dominguez,NYY,22,114,369,415,94,66,...,112.1,129,0.496,260,0.122,0.248,0.237,0.366,0.304,0.5


In [ ]:
import numpy as np
import pandas as pd
from itertools import combinations, permutations
from concurrent.futures import ThreadPoolExecutor
import warnings

warnings.filterwarnings('ignore')

# Provided run expectancy matrix
base_matrix = np.array([
    [0.481, 0.254, 0.095],  # Bases empty
    [0.831, 0.508, 0.224],  # Runner on 1st
    [1.088, 0.699, 0.348],  # Runner on 2nd
    [1.437, 0.953, 0.478],  # Runners on 1st & 2nd
    [1.353, 0.897, 0.382],  # Runner on 3rd
    [1.784, 1.141, 0.511],  # Runners on 1st & 3rd
    [2.052, 1.352, 0.661],  # Runners on 2nd & 3rd
    [2.417, 1.596, 0.815]   # Bases loaded
])

base_states = [
    (0, 0, 0), (1, 0, 0), (0, 1, 0), (1, 1, 0),
    (0, 0, 1), (1, 0, 1), (0, 1, 1), (1, 1, 1)
]

def get_player_probs(player_row):
    pa = max(player_row['PA'], 1)
    probs = {
        'single': player_row['1B'] / pa,
        'double': player_row['2B'] / pa,
        'triple': player_row['3B'] / pa,
        'hr': player_row['HR'] / pa,
        'walk': player_row['BB'] / pa,
        'hbp': player_row['HBP'] / pa,
        'strikeout': player_row['SO'] / pa,
        'out': max(0, 1 - (player_row['H'] + player_row['BB'] + player_row['HBP']) / pa)
    }
    if pa < 50:
        probs['single'] = 0.7 * probs['single'] + 0.3 * player_row['xBA']
        probs['double'] = 0.7 * probs['double'] + 0.3 * (player_row['xSLG'] - player_row['xBA']) / 2
        probs['hr'] = 0.7 * probs['hr'] + 0.3 * (player_row['xSLG'] - player_row['xBA']) / 4
    total = sum(probs.values())
    if total > 0:
        probs = {k: v / total for k, v in probs.items()}
    else:
        probs = {k: 1/8 for k in probs}
    return probs

def simulate_pa(player_probs):
    outcomes = list(player_probs.keys())
    probs = list(player_probs.values())
    return np.random.choice(outcomes, p=probs)

def update_state(bases, outs, outcome, player_row):
    runs = 0
    new_bases = bases.copy()
    new_outs = outs
    if outcome == 'hr':
        runs += sum(bases) + 1
        new_bases = [0, 0, 0]
    elif outcome == 'single':
        if bases[2]: runs += 1
        if bases[1]: new_bases[2] = 1
        if bases[0]: new_bases[1] = 1
        new_bases[0] = 1
    elif outcome == 'double':
        if bases[2]: runs += 1
        if bases[1]: runs += 1
        if bases[0]: new_bases[2] = 1
        new_bases[1] = 1
        new_bases[0] = 0
    elif outcome == 'triple':
        runs += sum(bases)
        new_bases = [0, 0, 1]
    elif outcome == 'walk' or outcome == 'hbp':
        if all(bases):
            runs += 1
        if bases[1]: new_bases[2] = 1
        if bases[0]: new_bases[1] = 1
        new_bases[0] = 1
    elif outcome == 'strikeout':
        new_outs += 1
    elif outcome == 'out':
        gdp_prob = player_row['GDP'] / max(player_row['PA'], 1)
        if bases[0] and outs < 2 and gdp_prob > np.random.random():
            new_outs = min(outs + 2, 3)
            new_bases[0] = 0
            if bases[1]: new_bases[1] = 0
            if bases[2]: new_bases[2] = 0
        else:
            new_outs += 1
    return new_bases, new_outs, runs

def simulate_inning(lineup, player_probs, batter_idx):
    outs = 0
    bases = [0, 0, 0]
    runs = 0
    while outs < 3:
        player = lineup[batter_idx % len(lineup)]
        outcome = simulate_pa(player_probs[player])
        bases, outs, new_runs = update_state(bases, outs, outcome, player_probs['rows'][player])
        runs += new_runs
        batter_idx += 1
    return runs, batter_idx

def simulate_game(lineup, player_probs, innings=9):
    total_runs = 0
    batter_idx = 0
    for _ in range(innings):
        runs, batter_idx = simulate_inning(lineup, player_probs, batter_idx)
        total_runs += runs
    return total_runs

def remove_duplicates_preserve_order(lineup):
    """Remove duplicate names while preserving order."""
    seen = set()
    result = []
    for player in lineup:
        if player not in seen:
            seen.add(player)
            result.append(player)
    return result

def evaluate_combination(combo, df, n_simulations=100):
    # Remove duplicates from combo first
    combo = remove_duplicates_preserve_order(combo)
    
    if len(combo) < 9:
        return combo, 0
    
    player_probs = {
        name: get_player_probs(df[df['Name'] == name].iloc[0])
        for name in combo
    }
    player_probs['rows'] = {name: df[df['Name'] == name].iloc[0] for name in combo}

    df_filtered = df[df['Name'].isin(combo)].copy()
    
    # Create combined metrics
    df_filtered['wOBA_plus_xSLG'] = df_filtered['wOBA'] + df_filtered['xSLG']
    df_filtered['OBP_plus_xBA'] = df_filtered['OBP'] + df_filtered['xBA']
    df_filtered['power_contact_score'] = df_filtered['xSLG'] + df_filtered['xBA']
    
    # Original system: pick leadoff at the end
    two_hole = df_filtered.nlargest(1, 'xwOBA')['Name'].iloc[0]
    power_hitters = df_filtered[~df_filtered['Name'].isin([two_hole])].nlargest(2, 'wOBA')['Name'].tolist()
    sluggers = df_filtered[~df_filtered['Name'].isin([two_hole] + power_hitters)].nlargest(2, 'wOBA_plus_xSLG')['Name'].tolist()
    leadoff = df_filtered[~df_filtered['Name'].isin([two_hole] + power_hitters + sluggers)].nlargest(1, 'OBP')['Name'].iloc[0]
    remaining = df_filtered[~df_filtered['Name'].isin([leadoff, two_hole] + power_hitters + sluggers)]['Name'].tolist()

    # Build lineup avoiding duplicates
    sorted_combo = [leadoff, two_hole] + power_hitters + sluggers + remaining
    sorted_combo = remove_duplicates_preserve_order(sorted_combo)[:9]
    
    if len(sorted_combo) < 9:
        remaining = [name for name in combo if name not in sorted_combo]
        sorted_combo += remaining[:9 - len(sorted_combo)]
        sorted_combo = remove_duplicates_preserve_order(sorted_combo)

    best_runs = 0
    best_lineup = sorted_combo

    # Test a subset of permutations
    for order in [sorted_combo] + list(permutations(sorted_combo))[:9]:
        runs = np.mean([simulate_game(list(order), player_probs) for _ in range(n_simulations)])
        if runs > best_runs:
            best_runs = runs
            best_lineup = order

    return best_lineup, best_runs

def optimize_lineup(df, n_simulations=100, n_players=14):
    # Validate dataframe
    if len(df) < n_players:
        raise ValueError(f"Dataframe has only {len(df)} players; need at least {n_players}.")
    if df['Name'].nunique() < n_players:
        raise ValueError(f"Dataframe has only {df['Name'].nunique()} unique players; need {n_players}.")

    required_cols = ['Name', 'PA', '1B', '2B', '3B', 'HR', 'BB', 'HBP', 'SO', 'GDP', 'wOBA', 'OBP', 'xBA', 'xSLG']
    missing_cols = [col for col in required_cols if col not in df.columns]
    if missing_cols:
        raise ValueError(f"Missing required columns: {missing_cols}")

    # Remove duplicates and filter
    df = df[df['PA'] > 0].drop_duplicates(subset='Name')
    if len(df) < n_players:
        raise ValueError(f"After filtering, only {len(df)} unique players remain; need {n_players}.")

    # Select top players by wOBA, ensuring no duplicates
    players = df.nlargest(n_players, 'wOBA')['Name'].tolist()
    players = remove_duplicates_preserve_order(players)

    # Generate all combinations
    combos = list(combinations(players, 9))
    print(f"Generated {len(combos)} combinations to evaluate")

    best_lineup = None
    best_runs = 0
    results = []

    # Parallel processing for combinations (using ThreadPoolExecutor for Windows/Jupyter compatibility)
    with ThreadPoolExecutor(max_workers=4) as executor:
        futures = [executor.submit(evaluate_combination, combo, df, n_simulations) for combo in combos]
        for i, future in enumerate(futures):
            try:
                lineup, runs = future.result()
                # Ensure no duplicates in final lineup
                lineup = remove_duplicates_preserve_order(lineup)
                results.append((lineup, runs))
                if runs > best_runs:
                    best_runs = runs
                    best_lineup = lineup
                if i % 100 == 0:
                    print(f"Processed {i+1}/{len(combos)} combinations")
            except Exception as e:
                print(f"Error processing combination {i+1}: {e}")
                continue

    # Sort results by runs, keep top 5
    results.sort(key=lambda x: x[1], reverse=True)
    top_lineups = [remove_duplicates_preserve_order(r[0]) for r in results[:5]]
    top_runs = [r[1] for r in results[:5]]

    return best_lineup, best_runs, top_lineups, top_runs

def plot_lineup_performance(lineups, runs):
    return {
        "type": "bar",
        "data": {
            "labels": [f"Lineup {i+1}" for i in range(len(lineups))],
            "datasets": [{
                "label": "Expected Runs",
                "data": runs,
                "backgroundColor": ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd"],
                "borderColor": ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd"],
                "borderWidth": 1
            }]
        },
        "options": {
            "scales": {
                "y": {
                    "beginAtZero": True,
                    "title": {"display": True, "text": "Expected Runs per Game"}
                },
                "x": {
                    "title": {"display": True, "text": "Lineup Configuration"}
                }
            },
            "plugins": {"legend": {"display": False}}
        }
    }

# Example usage
try:
    print("Starting lineup optimization...")
    best_lineup, expected_runs, tested_lineups, runs_list = optimize_lineup(NYY_players, n_simulations=50, n_players=9)
    print(f"\nOptimal Lineup: {best_lineup}\nExpected Runs: {expected_runs:.2f}")
    print("\nTop 5 Lineups and Runs:")
    for i, (lineup, runs) in enumerate(zip(tested_lineups, runs_list)):
        print(f"Lineup {i+1}: {lineup} -> {runs:.2f} runs")
    print("\nChart Configuration:")
    print(plot_lineup_performance(tested_lineups, runs_list))
except ValueError as e:
    print(f"Error: {e}")
except Exception as e:
    print(f"Unexpected error: {e}")
    import traceback
    traceback.print_exc()

Starting lineup optimization...
Generated 1 combinations to evaluate
Processed 1/1 combinations

Optimal Lineup: ['Cody Bellinger', 'Aaron Judge', 'Giancarlo Stanton', 'Trent Grisham', 'Ben Rice', 'Austin Wells', 'Jazz Chisholm Jr.', 'Jasson Dominguez', 'Paul Goldschmidt']
Expected Runs: 3.96

Top 5 Lineups and Runs:
Lineup 1: ['Cody Bellinger', 'Aaron Judge', 'Giancarlo Stanton', 'Trent Grisham', 'Ben Rice', 'Austin Wells', 'Jazz Chisholm Jr.', 'Jasson Dominguez', 'Paul Goldschmidt'] -> 3.96 runs

Chart Configuration:
{'type': 'bar', 'data': {'labels': ['Lineup 1'], 'datasets': [{'label': 'Expected Runs', 'data': [np.float64(3.96)], 'backgroundColor': ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd'], 'borderColor': ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd'], 'borderWidth': 1}]}, 'options': {'scales': {'y': {'beginAtZero': True, 'title': {'display': True, 'text': 'Expected Runs per Game'}}, 'x': {'title': {'display': True, 'text': 'Lineup Configuration'}}}, 'plugins

In [33]:
!pip install discord.py nest_asyncio


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\miraj\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
import discord
from discord.ext import commands
import nest_asyncio
import asyncio

# Apply nest_asyncio to handle Colab's event loop
nest_asyncio.apply()

# Bot setup
intents = discord.Intents.default()
intents.message_content = True
bot = commands.Bot(command_prefix='!', intents=intents)

@bot.event
async def on_ready():
    print(f"Bot logged in as {bot.user.name}")

@bot.command()
async def lineup(ctx, team: str):
    try:
        if team not in ['ARI', 'ATL', 'BAL', 'BOS', 'CHC', 'CWS', 'CIN', 'CLE',
                        'COL', 'DET', 'HOU', 'KC', 'LAA', 'LAD', 'MIA', 'MIL',
                        'MIN', 'NYM', 'NYY', 'OAK', 'PHI', 'PIT', 'SD', 'SF',
                        'SEA', 'STL', 'TB', 'TEX', 'TOR', 'WSN']:
                        await ctx.send("Invalid team abbreviation. Please enter a valid team abbreviation.")

        # Assume NYY_players is already loaded in the Colab environment
        else:
          await ctx.send("Generating lineup...")

          # Split output if too long for Discord (2000 char limit)
          teams_players = filter_by_team(test_df, team)
          best_lineup, expected_runs, tested_lineups, runs_list = optimize_lineup(teams_players, n_simulations=100, n_players=9)
          output = ""
          for i, player in enumerate(best_lineup):
            output += (f"{i+1}. {player}\n")

          await ctx.send(f"Optimal Lineup: \n{output}\nExpected Runs: {expected_runs:.2f}")
    except Exception as e:
        await ctx.send(f"Error: {str(e)}")


# Run bot (replace 'YOUR_TOKEN' with your Discord bot token)
bot.run('')

[2025-09-07 22:14:08] [INFO    ] discord.client: logging in using static token
[2025-09-07 22:14:08] [INFO    ] discord.client: logging in using static token
[2025-09-07 22:14:09] [INFO    ] discord.gateway: Shard ID None has connected to Gateway (Session ID: f688b69759bf059fb4ba19dc330499be).
[2025-09-07 22:14:09] [INFO    ] discord.gateway: Shard ID None has connected to Gateway (Session ID: f688b69759bf059fb4ba19dc330499be).


Bot logged in as Lineup Bot
Generated 1 combinations to evaluate
Processed 1/1 combinations
Generated 1 combinations to evaluate
Processed 1/1 combinations
Generated 1 combinations to evaluate
Processed 1/1 combinations
Generated 1 combinations to evaluate
Processed 1/1 combinations


[2025-09-07 23:19:31] [ERROR   ] discord.client: Attempting a reconnect in 0.47s
Traceback (most recent call last):
  File "C:\Users\miraj\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\aiohttp\connector.py", line 1532, in _create_direct_connection
    hosts = await self._resolve_host(host, port, traces=traces)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\miraj\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\aiohttp\connector.py", line 1148, in _resolve_host
    return await asyncio.shield(resolved_host_task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\asyncio\futures.py", line 287, in __await__
    yield self  # This tells Task to wait for completion.
    ^^^^^^^^^^
  File "C:\Program

Bot logged in as Lineup Bot
Generated 1 combinations to evaluate
Processed 1/1 combinations


In [ ]:
""

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,...,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA,L-WAR
0,15640,2024,Aaron Judge,NYY,32,158,559,704,180,85,...,117.5,238,0.609,391,0.146,0.267,0.310,0.723,0.479,11.6
3,20123,2024,Juan Soto,NYY,25,157,576,713,166,90,...,115.7,262,0.568,461,0.189,0.263,0.316,0.646,0.462,8.1
185,27562,2024,Austin Wells,NYY,24,115,354,414,81,49,...,111.2,107,0.389,275,0.134,0.255,0.242,0.429,0.339,3.7
304,27647,2024,Anthony Volpe,NYY,23,160,637,689,155,109,...,108.7,170,0.349,487,0.186,0.285,0.238,0.332,0.278,2.9
193,16997,2024,Gleyber Torres,NYY,27,154,587,665,151,110,...,111.2,163,0.354,461,0.164,0.258,0.240,0.368,0.307,1.7
